In [1]:
import os

from bids import BIDSLayout
from nipype.pipeline import engine as pe
from nipype.interfaces import mrtrix3, fsl, utility as niu

In [2]:
def init_bet_frac_wf(frac=0.3):
    wf = pe.Workflow(name='bet_frac_wf')
    
    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_file',
                                                      'bvec_file',
                                                      'bval_file']), 
                        name='inputnode')
    
    outputnode = pe.Node(niu.IdentityInterface(fields=['out_file']), name='outputnode')

    def b0_average(in_dwi, in_bval, b0_thresh=0, out_file=None):

        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        if out_file is None:
            out_file = fname_presuffix(
                in_dwi, suffix='_avg_b0', newpath=os.path.abspath('.')
            )

        imgs = np.array(nib.four_to_three(nib.load(in_dwi, mmap=NUMPY_MMAP)))
        bval = np.loadtxt(in_bval)
        index = np.argwhere(bval <= b0_thresh).flatten().tolist()

        b0s = [im.get_data().astype(np.float32) for im in imgs[index]]
        b0 = np.average(np.array(b0s), axis=0)

        hdr = imgs[0].header.copy()
        hdr.set_data_shape(b0.shape)
        hdr.set_xyzt_units('mm')
        hdr.set_data_dtype(np.float32)
        nib.Nifti1Image(b0, imgs[0].affine, hdr).to_filename(out_file)
        return out_file

    avg_b0_0 = pe.Node(
        niu.Function(
            input_names=['in_dwi', 'in_bval', 'b0_thresh'],
            output_names=['out_file'],
            function=b0_average),
        name='b0_avg_pre')

    bet_dwi0 = pe.Node(
        fsl.BET(frac=frac, mask=True, robust=True),
        name='bet_dwi_pre')
    
    wf.connect(
        [(inputnode, avg_b0_0, [('dwi_file', 'in_dwi'),
                                ('bval_file', 'in_bval')]),
         (avg_b0_0, bet_dwi0, [('out_file', 'in_file')]),
         (bet_dwi0, outputnode, [('out_file', 'out_file')])
        ]
    )
    
    return wf

In [3]:
def init_bet_func_wf():
    wf = pe.Workflow(name='bet_func_wf')
    
    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_file',
                                                      'bvec_file',
                                                      'bval_file']), 
                        name='inputnode')
    
    outputnode = pe.Node(niu.IdentityInterface(fields=['out_file']), name='outputnode')

    def b0_average(in_dwi, in_bval, b0_thresh=0, out_file=None):

        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        if out_file is None:
            out_file = fname_presuffix(
                in_dwi, suffix='_avg_b0', newpath=os.path.abspath('.')
            )

        imgs = np.array(nib.four_to_three(nib.load(in_dwi, mmap=NUMPY_MMAP)))
        bval = np.loadtxt(in_bval)
        index = np.argwhere(bval <= b0_thresh).flatten().tolist()

        b0s = [im.get_data().astype(np.float32) for im in imgs[index]]
        b0 = np.average(np.array(b0s), axis=0)

        hdr = imgs[0].header.copy()
        hdr.set_data_shape(b0.shape)
        hdr.set_xyzt_units('mm')
        hdr.set_data_dtype(np.float32)
        nib.Nifti1Image(b0, imgs[0].affine, hdr).to_filename(out_file)
        return out_file

    avg_b0_0 = pe.Node(
        niu.Function(
            input_names=['in_dwi', 'in_bval', 'b0_thresh'],
            output_names=['out_file'],
            function=b0_average),
        name='b0_avg_pre')

    bet_dwi0 = pe.Node(
        fsl.BET(mask=True, functional=True),
        name='bet_dwi_pre')

    wf.connect(
        [(inputnode, avg_b0_0, [('dwi_file', 'in_dwi'),
                                ('bval_file', 'in_bval')]),
         (avg_b0_0, bet_dwi0, [('out_file', 'in_file')]),
         (bet_dwi0, outputnode, [('out_file', 'out_file')])
        ]
    )
    
    return wf

In [4]:
def init_median_otsu_wf():
    wf = pe.Workflow(name='median_otsu_wf')
    
    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_file',
                                                      'bvec_file',
                                                      'bval_file']), 
                        name='inputnode')
    
    outputnode = pe.Node(niu.IdentityInterface(fields=['out_file']), name='outputnode')

    def b0_average(in_dwi, in_bval, b0_thresh=0, out_file=None):

        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        if out_file is None:
            out_file = fname_presuffix(
                in_dwi, suffix='_avg_b0', newpath=os.path.abspath('.')
            )

        imgs = np.array(nib.four_to_three(nib.load(in_dwi, mmap=NUMPY_MMAP)))
        bval = np.loadtxt(in_bval)
        index = np.argwhere(bval <= b0_thresh).flatten().tolist()

        b0s = [im.get_data().astype(np.float32) for im in imgs[index]]
        b0 = np.average(np.array(b0s), axis=0)

        hdr = imgs[0].header.copy()
        hdr.set_data_shape(b0.shape)
        hdr.set_xyzt_units('mm')
        hdr.set_data_dtype(np.float32)
        nib.Nifti1Image(b0, imgs[0].affine, hdr).to_filename(out_file)
        return out_file

    avg_b0_0 = pe.Node(
        niu.Function(
            input_names=['in_dwi', 'in_bval', 'b0_thresh'],
            output_names=['out_file'],
            function=b0_average),
        name='b0_avg_pre')
    
    def get_b0_mask_fn(b0_file):
        import os
        import nibabel as nib
        from nipype.utils.filemanip import fname_presuffix
        from dipy.segment.mask import median_otsu

        mask_file = fname_presuffix(
            b0_file, suffix='_mask', newpath=os.path.abspath('.')
        )
        img = nib.load(b0_file)
        data, aff = img.get_data(), img.affine
        brain, mask = median_otsu(data, 2, 1)
        nib.Nifti1Image(mask.astype(float), aff).to_filename(mask_file)
        return brain, mask_file
    
    b0mask_node = pe.Node(
        niu.Function(
            input_names=['b0_file'],
            output_names=['brain_file', 'mask_file'],
            function=get_b0_mask_fn),
        name='getB0Mask')

    wf.connect(
        [(inputnode, avg_b0_0, [('dwi_file', 'in_dwi'),
                                ('bval_file', 'in_bval')]),
         (avg_b0_0, b0mask_node, [('out_file', 'b0_file')]),
         (b0mask_node, outputnode, [('brain_file', 'out_file')])
        ]
    )
    
    return wf

In [5]:
def init_dwi2mask_wf():
    wf = pe.Workflow(name='dwi2mask_wf')
    
    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_file',
                                                      'bvec_file',
                                                      'bval_file']), 
                        name='inputnode')
    
    outputnode = pe.Node(niu.IdentityInterface(fields=['out_file']), name='outputnode')
  
    bias_correct = pe.Node(mrtrix3.DWIBiasCorrect(use_ants=True), name='bias_correct')
    
    b0mask_node = pe.Node(mrtrix3.BrainMask(), name='b0mask_node')
    
    wf.connect(
        [(inputnode, bias_correct, [('dwi_file', 'in_file'),
                                    ('bvec_file', 'in_bvec'),
                                    ('bval_file', 'in_bval')]),
         (bias_correct, b0mask_node, [('out_file', 'in_file')]),
         (inputnode, b0mask_node, [('bvec_file', 'in_bvec'),
                                   ('bval_file', 'in_bval')]),
         (b0mask_node, outputnode, [('out_file', 'out_file')])
        ]
    )
    
    return wf

In [6]:
data_dir = os.path.abspath('../data')
layout = BIDSLayout(data_dir)

In [7]:
dwi_files = layout.get(datatype='dwi', extension=['nii.gz', 'nii'], return_type='file')
dwi_file = dwi_files[0]
bvec_file = layout.get_bvec(dwi_file)
bval_file = layout.get_bval(dwi_file)

In [8]:
workflows = [init_bet_frac_wf(), init_bet_func_wf(), init_median_otsu_wf(), init_dwi2mask_wf()]

In [ ]:
mrtrix3.BrainMask()

In [9]:
for test_wf in workflows:
    test_wf.base_dir = os.path.join(os.getcwd(), 'skullstrip')

    inputspec = test_wf.get_node('inputnode')
    inputspec.inputs.dwi_file = dwi_file
    inputspec.inputs.bval_file = bval_file
    inputspec.inputs.bvec_file = bvec_file

    test_wf.write_graph(graph2use='colored')
    test_wf.config['execution']['remove_unnecessary_outputs'] = False
    test_wf.config['execution']['keep_inputs'] = True
    test_wf.config['execution']['crashfile_format'] = 'txt'

    test_wf.run()

190830-16:43:00,34 nipype.workflow INFO:
	 Generated workflow graph: /mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/notebooks/skullstrip/bet_frac_wf/graph.png (graph2use=colored, simple_form=True).
190830-16:43:00,173 nipype.workflow INFO:
	 Workflow bet_frac_wf settings: ['check', 'execution', 'logging', 'monitoring']
190830-16:43:00,208 nipype.workflow INFO:
	 Running serially.
190830-16:43:00,210 nipype.workflow INFO:
	 [Node] Setting-up "bet_frac_wf.b0_avg_pre" in "/mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/notebooks/skullstrip/bet_frac_wf/b0_avg_pre".
190830-16:43:00,219 nipype.workflow INFO:
	 [Node] Cached "bet_frac_wf.b0_avg_pre" - collecting precomputed outputs
190830-16:43:00,221 nipype.workflow INFO:
	 [Node] "bet_frac_wf.b0_avg_pre" found cached.
190830-16:43:00,222 nipype.workflow INFO:
	 [Node] Setting-up "bet_frac_wf.bet_dwi_pre" in "/mnt/tigrlab/projects/mjoseph/pipelines/dmriprep-notebooks/notebooks/skullstrip/bet_frac_wf/bet_dwi_pre".
19083

RuntimeError: Workflow did not execute cleanly. Check log for details